<a href="https://colab.research.google.com/github/Busettyvivek/Machine-Learning-Projects/blob/main/Movie_Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# Importing dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [53]:
df=pd.read_csv('movies recommendation.csv')

In [54]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [55]:
df.shape

(4803, 24)

In [56]:
df.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [57]:
df=df.fillna(' ')

In [58]:
df.isna().sum()

index                   0
budget                  0
genres                  0
homepage                0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
cast                    0
crew                    0
director                0
dtype: int64

In [59]:
selected_features =['genres', 'keywords','tagline','cast','director']
df['content']=df['genres']+' '+df['keywords']+' '+ df['tagline']+ ' '+df['cast']+' '+df['director']

In [60]:
df['content']

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance   A newlywed couple's honeymoon...
4800    Comedy Drama Romance TV Movie date love at fir...
4801        A New Yorker in Shanghai Daniel Henney Eli...
4802    Documentary obsession camcorder crush dream gi...
Name: content, Length: 4803, dtype: object

In [61]:
# stemming (word ---> rootword)
port_stem=PorterStemmer()

In [62]:
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords # removes unnecessary words
from nltk.stem.porter import PorterStemmer # gives the root word

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
def stemming(content):
  stem_content=re.sub('[^a-zA-Z]',' ',content) # it basically takes only alphabets(lower,upper) and excludes other special characters and symbols
  stem_content=stem_content.lower()
  stem_content=stem_content.split()
  stem_content=[port_stem.stem(word) for word in stem_content if not word in stopwords.words('english')]
  stem_content =" ".join(stem_content)
  return stem_content

In [64]:
df['content']=df['content'].apply(stemming)

In [65]:
x=df['content']

In [66]:
x

0       action adventur fantasi scienc fiction cultur ...
1       adventur fantasi action ocean drug abus exot i...
2       action adventur crime spi base novel secret ag...
3       action crime drama thriller dc comic crime fig...
4       action adventur scienc fiction base novel mar ...
                              ...                        
4798    action crime thriller unit state u mexico barr...
4799    comedi romanc newlyw coupl honeymoon upend arr...
4800    comedi drama romanc tv movi date love first si...
4801    new yorker shanghai daniel henney eliza coup b...
4802    documentari obsess camcord crush dream girl dr...
Name: content, Length: 4803, dtype: object

In [67]:
tf=TfidfVectorizer()
tf.fit(x)
x=tf.transform(x)

In [70]:
print(x)

  (0, 15139)	0.2052522310553095
  (0, 14871)	0.24026470377934847
  (0, 14863)	0.12875501676155404
  (0, 14587)	0.20164827377007877
  (0, 14523)	0.12740774864377094
  (0, 13033)	0.15213516971481178
  (0, 12829)	0.34513124291779446
  (0, 12728)	0.21738843062624963
  (0, 12549)	0.20929853505545373
  (0, 12131)	0.1053209249350964
  (0, 11900)	0.15264686191548815
  (0, 11874)	0.22127568137478468
  (0, 11596)	0.19738383622085057
  (0, 10240)	0.27652273888326423
  (0, 9066)	0.15239002415366618
  (0, 7728)	0.2052522310553095
  (0, 6793)	0.11463157223101918
  (0, 5083)	0.16734367774450565
  (0, 4735)	0.10471198725789553
  (0, 4580)	0.11273156644207886
  (0, 4305)	0.24415195452788352
  (0, 3161)	0.21560402385508065
  (0, 2782)	0.24855895595249378
  (0, 2641)	0.22568268279939493
  (0, 2071)	0.17552313506858697
  :	:
  (4801, 10378)	0.22124713594289835
  (4801, 9715)	0.1385948353997998
  (4801, 6372)	0.31053778940920307
  (4801, 6077)	0.2962036112137938
  (4801, 4209)	0.2536402429374028
  (4801, 3

In [71]:
# cosine similarity
similarity=cosine_similarity(x)

In [72]:
print(similarity)

[[1.         0.04757067 0.04209981 ... 0.         0.         0.        ]
 [0.04757067 1.         0.03791066 ... 0.01223813 0.         0.        ]
 [0.04209981 0.03791066 1.         ... 0.         0.0520925  0.        ]
 ...
 [0.         0.01223813 0.         ... 1.         0.         0.02762031]
 [0.         0.         0.0520925  ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02762031 0.         1.        ]]


In [73]:
similarity.shape

(4803, 4803)

In [75]:
# Getting the movie name from the user
movie_name=input('Enter a movie name:')

Enter a movie name:iron man


In [76]:
# Creating a list with all movie names given in the dataset
list_of_all_titles=df['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [77]:
# find the close match for the movie name given by the user
find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)
find_close_match


['Iron Man', 'Iron Man 3', 'Iron Man 2']

In [78]:
close_match =find_close_match[0]
close_match

'Iron Man'

In [80]:
index_of_the_movie=df[df.title==close_match]['index'].values[0]
index_of_the_movie


68

In [85]:
# getting a list of similar movies based on the index
similarity_score=list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.036279148864686264), (1, 0.061558683705088595), (2, 0.016092132930621707), (3, 0.04036790530962703), (4, 0.03670710172252939), (5, 0.014984720198289548), (6, 0.04056023641993075), (7, 0.2559459585117725), (8, 0.00862439867233422), (9, 0.10665875073850486), (10, 0.11445692316127393), (11, 0.014425658558468868), (12, 0.014745006244598811), (13, 0.01374419711272003), (14, 0.1344353775963146), (15, 0.019265004671949193), (16, 0.24471600783867883), (17, 0.014469193298354421), (18, 0.03927896524269903), (19, 0.08522031330525329), (20, 0.08335695307236371), (21, 0.01296685602249992), (22, 0.007600482681465085), (23, 0.007770423866269054), (24, 0.013990230591202722), (25, 0.0), (26, 0.23252314013567693), (27, 0.033647154746330246), (28, 0.06609997435292247), (29, 0.015891760262266477), (30, 0.1142378515579299), (31, 0.3467917032540424), (32, 0.01894997570690099), (33, 0.1365048048461158), (34, 0.0), (35, 0.03827407061606145), (36, 0.036091090604745774), (37, 0.008935570647780907), (38, 

In [86]:
# sort the movies based on their simillarity score
sorted_similarity_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_similarity_movies)

[(68, 1.0), (79, 0.4970792262992668), (31, 0.3467917032540424), (7, 0.2559459585117725), (16, 0.24471600783867883), (26, 0.23252314013567693), (85, 0.20455741986311915), (182, 0.19536496183133012), (511, 0.17533432290975315), (3623, 0.17341034843112504), (203, 0.16813087028510112), (64, 0.1656379047858398), (788, 0.1646084646483868), (174, 0.16083017638989086), (101, 0.15957134204907109), (4401, 0.15440968284752232), (1740, 0.14642325166193293), (126, 0.1416030302429484), (33, 0.1365048048461158), (46, 0.13603580281799374), (14, 0.1344353775963146), (2487, 0.13414409901343505), (783, 0.13341389571369594), (169, 0.13230080991980564), (870, 0.1318121155980149), (138, 0.13015916542736053), (2442, 0.12899058206975336), (94, 0.12788866195702808), (2651, 0.12785910400314024), (661, 0.12614513974414276), (1192, 0.12550061387481526), (607, 0.12455949094993161), (131, 0.1234444996759791), (38, 0.12023427123751528), (1553, 0.11849950588901001), (1368, 0.11459406557071378), (10, 0.114456923161273

In [89]:
print("Movies suggested based on your intrest : \n")
i=1
for movie in sorted_similarity_movies:
  index=movie[0]
  title_from_index=df[df.index==index]['title'].values[0]
  if i<30:
    print(i,'.',title_from_index)
    i=i+1


Movies suggested based on your intrest : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . Made
11 . X2
12 . X-Men: Apocalypse
13 . Deadpool
14 . The Incredible Hulk
15 . X-Men: First Class
16 . The Helix... Loaded
17 . Kick-Ass 2
18 . Thor: The Dark World
19 . X-Men: The Last Stand
20 . X-Men: Days of Future Past
21 . Man of Steel
22 . Duets
23 . Mortdecai
24 . Captain America: The First Avenger
25 . Superman II
26 . The Last Airbender
27 . Southland Tales
28 . Guardians of the Galaxy
29 . The Good Night
